# TDA@HSE+YSDA, 2024

## Seminar 9

In [ ]:
!pip install --upgrade Cython
!pip install --upgrade Ripser
!pip install --upgrade diagram2vec
!pip install --upgrade giotto-tda

In [ ]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

from ripser import lower_star_img
from ripser import Rips
vr = Rips()
from gtda.homology import VietorisRipsPersistence

import persim
import diagram2vec

from scipy.ndimage import gaussian_filter

from sklearn.datasets import make_circles
from sklearn.manifold import MDS

from gtda.diagrams import PersistenceEntropy, PersistenceImage, BettiCurve

import pickle
from tqdm import tqdm

import torch
from torch.nn import Linear
from torch.nn.functional import relu

from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from sklearn.model_selection import cross_val_score

In [ ]:
def diagram_reshape(diagram):
    zero_idx = np.where(diagram[:,2]==0)
    one_idx = np.where(diagram[:,2]==1)
    return diagram[zero_idx], diagram[one_idx]

## 1. Persistent diagrams, Wasserstein distance and stability

In [ ]:
# original data
X, y = make_circles(n_samples=200, noise=0.1)
X = X[y==0]

Topology studies data invariant to continous transformations, so topological invariants like (persistent) homology will not change under such class of transformations.

**Exercise**

Apply rotation and dilation transformations to copy of original data.

In [ ]:
theta = np.radians(30)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c,-s), (s, c)))
print(R)

In [ ]:
# transformed data
X_transformed = np.copy(X)
X_transformed[:,0] = X[:,0] * 0.75
X_transformed = np.dot(X_transformed, R)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

ax[0].set_title("Data")
ax[0].set_xlim(-1.5, 1.5)
ax[0].set_ylim(-1.5, 1.5)
ax[0].grid(linestyle="dotted")
ax[0].scatter(X[:,0], X[:,1], c="b", alpha=0.5)

ax[1].set_title("Transformed data")
ax[1].set_xlim(-1.5, 1.5)
ax[1].set_ylim(-1.5, 1.5)
ax[1].grid(linestyle="dotted")
ax[1].scatter(X_transformed[:,0], X_transformed[:,1], c="b", alpha=0.5)

plt.show()

**Exercise**

Compute persistence diagrams of a filtration of Vietoris-Rips complex built on point cloud data

In [ ]:
diagram = vr.fit_transform(X)
diagram_transformed = vr.fit_transform(X_transformed)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
plt.suptitle("Persistent diagram of a filtration")

ax[0].set_title("Data")
ax[0].grid(linestyle="dotted")
vr.plot(diagram, ax=ax[0])

ax[1].set_title("Transformed data")
ax[1].grid(linestyle="dotted")
vr.plot(diagram_transformed, ax=ax[1])

plt.show()

One can define the geometry on the space of persistent diagrams, defining a metric on it. Optimal transport approach is used to compare persistent diagrams which are multisets of intervals of arbitrary cardinality.  

The variants of optimal transport distances are _Wasserstein-2 distance_, and its approximations like _sliced Wasserstein distance_ and _Bollteneck distance_, which is Wasserstein-$\infty$ distance.

**Exercise**

Compute Bottleneck `persim.bottleneck` and sliced Wasserstein distances `persim.sliced_wasserstein` between perisistent diagrams of original and transformed data.

In [ ]:
diagram

In [ ]:
diagram[1].shape

In [ ]:
diagram_transformed[1].shape

In [ ]:
persim.bottleneck(diagram[1], diagram_transformed[1]) # pc w noise=0.1 vs w/ noise 0.12

In [ ]:
persim.sliced_wasserstein(diagram[1], diagram_transformed[1])

Bottleneck distance used a single matching between most discriminative pair of points.

**Exercise**

Visualize Bottleneck matching.

In [ ]:
# compute Bottleneck distance matching
d, matching = persim.bottleneck(diagram[1], diagram_transformed[1], matching=True)

In [ ]:
# plot Bottleneck distance matchign
fig, ax = plt.subplots(figsize=(10, 5))
plt.suptitle("Bottleneck distance matching")
ax.grid(linestyle="dotted")
persim.bottleneck_matching(diagram[1], diagram_transformed[1], matching, labels=['Original $H_1$', 'Transformed $H_1$'], ax=ax)
plt.show()

For Bottleneck distance stability to small perturbations is theoretically proved.

**Exercise**

Plot Bottleneck distance with respect to different level of Gaussian noise applied to original data.

In [ ]:
n_samples = 200

n_noise_levels = 11
n_repeats = 10

# original data
X_orig, y = make_circles(n_samples=n_samples, noise=0.0)
X_orig = X_orig
diagram_orig = vr.fit_transform(X_orig)

distances = np.zeros((n_noise_levels, n_repeats))

for j in tqdm(range(n_repeats)):
    for i, noise_level in enumerate(np.linspace(0, 0.2, n_noise_levels)):

        X_noisy, _ = make_circles(n_samples=n_samples, noise=noise_level)
        X_noisy = X_noisy

        diagram_noise = vr.fit_transform(X_noisy)
        
        distances[i,j] = persim.bottleneck(diagram_orig[1], diagram_noise[1])

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.set_xlabel("Noise level")
ax.set_ylabel("Bottleneck distance")
ax.set_xticks(range(n_noise_levels), np.linspace(0, 0.2, n_noise_levels))
ax.grid(linestyle="dotted")
ax.plot(distances.mean(axis=1), c="b")

std = distances.std(axis=1)
lower = distances.mean(axis=1) - std
upper = distances.mean(axis=1) + std
ax.fill_between(range(n_noise_levels), lower, upper, color='b', alpha=.2)

plt.show()

## 3. Persistent homology of graphs

Pipeline is as follows:

1. compute persistent diagrams via Ripser 
2. compute vectorization of diagrams, so-called persistent images and Betti curves
3. apply classifier on vectorization

In [ ]:
# load data
X_graphs = pickle.load(open("./data/metric_graphs/X.pkl", "rb"))
y_all = pickle.load(open("./data/metric_graphs/y_all.pkl", "rb"))
y_dnod = pickle.load(open("./data/metric_graphs/y_d_nod.pkl", "rb"))

y_col = ["a"] * len(y)
y_col = np.array(y_col)

y_col[y==0] = "blue"
y_col[y==2] = "green"
y_col[y==1] = "red"
y_col[y==3] = "yellow"

### Compute persistent diagrams

In [ ]:
# add h_1 diagrams only
maxdim = 1
h = 1

rips = Rips(maxdim=maxdim)

diagrams = []
for x in X_graphs:
    diagrams.append(rips.fit_transform(x, distance_matrix=True)[h])

len(diagrams)

### Clusterization

In [ ]:
%%time
n = len(X_graphs)
distances = np.zeros((n, n))

for i in tqdm(range(0, n)):
    for j in range(i+1, n):
        distances[i,j] = persim.sliced_wasserstein(diagrams[i], diagrams[j])
        
distances_symmetrize = distances + distances.T

In [ ]:
distances_symmetrize = distances + distances.T

In [ ]:
mds = MDS(n_components=2, max_iter=3000, eps=1e-9, dissimilarity="precomputed", random_state=1, n_jobs=-1)
X_metric = mds.fit(distances_symmetrize).embedding_

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")

plt.scatter(X_metric[:, 0], X_metric[:, 1], c=y_all)
plt.show()

### Vectorization

Persistent diagram is a multiset of intervals of arbitrary length which is can not be handled by methods of machine learning. One possible to solutions besides providing a metric on the space of persistent diagrams is vectorization of diagrams to a vector of fixed length.

In [ ]:
vr = VietorisRipsPersistence()
diagrams = vr.fit_transform(X_graphs)

#### Betti curve

In [ ]:
n_bins_curve = 50

betti_curves = BettiCurve(n_bins=n_bins_curve).fit_transform(diagrams)
betti_curves.shape

#### Persistence image

In [ ]:
n_bins_image = 15

persistence_images = PersistenceImage(sigma=0.1, n_bins=n_bins_image).fit_transform(diagrams)
persistence_images.shape

In [ ]:
idx = 4

diagram0, diagram1 = diagram_reshape(diagrams[idx])

fig, ax = plt.subplots(ncols=3, figsize=(13.5, 4), dpi=200)
ax[0].set_title("Persistence diagram")
ax[1].set_title("Persistence image, dim=1")
ax[2].set_title("Betti curve")

# persistence diagram
ax[0].scatter(diagram0[:,0], diagram0[:,1], s=10, c="r")
ax[0].scatter(diagram1[:,0], diagram1[:,1], s=10, c="b")

# persistence image
ax[1].imshow(persistence_images[idx][1])

# Betti curve
ax[2].step(np.linspace(0,1,50), betti_curves[idx,0], color="r", where="post", linewidth=2, label="dim=0")
ax[2].step(np.linspace(0,1,50), betti_curves[idx,1], color="b", where="post", linewidth=2, label="dim=1")
ax[2].legend(fontsize=12)

plt.show()

### Classification

#### Persistence images

In [ ]:
X_control = persistence_images[y_all==0]
X_depression = persistence_images[y_all==1]
X = np.concatenate((X_control, X_depression), axis=0)
X0 = X[:,0].reshape(-1, n_bins_image**2)
X1 = X[:,1].reshape(-1, n_bins_image**2)
y = np.concatenate((np.zeros(25), np.ones(25)), axis=0)

In [ ]:
clf = LogisticRegression(penalty="l2", C=5.0, solver="liblinear", random_state=42)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

In [ ]:
acc_pi_h0 = cross_val_score(clf, X0, y, cv=cv)
print("Accuracy, PI, H0: {:.4f} ± {:.4f}".format(np.mean(acc_pi_h0), np.std(acc_pi_h0)))

In [ ]:
acc_pi_h1 = cross_val_score(clf, X1, y, cv=cv)
print("Accuracy, PI, H1: {:.4f} ± {:.4f}".format(np.mean(acc_pi_h1), np.std(acc_pi_h1)))

#### Betti curves

In [ ]:
X_control = betti_curves[y_all==0]
X_depression = betti_curves[y_all==1]
X = np.concatenate((X_control, X_depression), axis=0)
X0 = X[:,0].reshape(-1, n_bins_curve)
X1 = X[:,1].reshape(-1, n_bins_curve)
y = np.concatenate((np.zeros(25), np.ones(25)), axis=0)

In [ ]:
clf = RandomForestClassifier(random_state=42)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

In [ ]:
acc_curve_h0 = cross_val_score(clf, X0, y, cv=cv)
print("Accuracy, Betti curve, H0: {:.4f} ± {:.4f}".format(np.mean(acc_curve_h0), np.std(acc_curve_h0)))

In [ ]:
acc_curve_h1 = cross_val_score(clf, X1, y, cv=cv)
print("Accuracy, Betti curve, H1: {:.4f} ± {:.4f}".format(np.mean(acc_curve_h1), np.std(acc_curve_h1)))

## 4. Persistent homology of digital images

Persistence Diagrams with Linear Machine Learning Models (Obayashi, Hiraoka), 2017  
https://arxiv.org/abs/1706.10082

In [ ]:
W = 300
sigma1 = 4
sigma2 = 2
t = 0.01

def generate(N, S, W=300, sigma1=4, sigma2=2, t=0.01, bins=64):

    z = np.zeros((N, S, 2))
    for n in range(N):
        z[n, 0] = np.random.uniform(0, W, size=(2))
        for s in range(S-1):
            d_1 = np.random.normal(0, sigma1)
            d_2 = np.random.normal(0, sigma1)
            z[n, s+1, 0] = (z[n, s, 0] + d_1) % W
            z[n, s+1, 1] = (z[n, s, 1] + d_2) % W

    z_r = z.reshape(N*S, 2)
    H, _, _ = np.histogram2d(z_r[:,0], z_r[:,1], bins=bins)
    
    G = gaussian_filter(H, sigma2)
    G[G < t] = 0
    
    return G

### Image generation

Generate 100 images accoring to model A and model B

In [ ]:
images = np.zeros((100,64,64))

# class A
N = 100
S = 30

for n in range(50):
    images[n] = generate(N, S)
    
# class B
N = 250
S = 10

for n in range(50):
    images[n+50] = generate(N, S)

In [ ]:
fig = plt.figure()
plt.gray()

ax1 = fig.add_subplot(121)
plt.title("Class A")

ax2 = fig.add_subplot(122)
plt.title("Class B")

ax1.imshow(images[int(np.random.uniform(0, 50))])
ax2.imshow(images[int(np.random.uniform(51, 100))])

plt.show()

### Compute persistent diagrams

In [ ]:
diags = []

for i in range(images.shape[0]):
    diags.append(lower_star_img(images[i])[:-1])

In [ ]:
plt.figure(figsize=(4,4))
persim.plot_diagrams(diags[52])

### Vectorization

In [ ]:
betti_curves = diagram2vec.persistence_curve(diags, m=25)
betti_curves.shape

### Classification

#### Betti curves

In [ ]:
X_betti_curves = betti_curves[0]
y = np.concatenate((np.zeros(50), np.ones(50)), axis=0)

y_col = ["b"] * len(y)
y_col = np.array(y_col)

y_col[y==1] = "r"

In [ ]:
clf = RandomForestClassifier(random_state=42)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

In [ ]:
acc_curve_h0 = cross_val_score(clf, X_betti_curves, y, cv=cv)
print("Accuracy, Betti curve, H0: {:.4f} ± {:.4f}".format(np.mean(acc_curve_h0), np.std(acc_curve_h0)))

### Clusterization

**Exercise**

Compute the two-dimensional embeddings using linear and nonlinear techniques learned during the course, given persistent images, Betti curves and pairwise distances between data points.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, SpectralEmbedding

In [ ]:
X_emb = Isomap().fit_transform(X_betti_curves)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")

plt.scatter(X_emb[:, 0], X_emb[:, 1], c=y_col)
plt.show()

## 5. Deep sets

#### Problem
Persistence diagram is a multiset of vectors $D = \{(b_i, d_i, h_i)\}_{i=1}^N$ where $b_i$, $d_i$ are the birth and death times of $i$-th topological feature of dimension $h_i$. The classic approach to introduce persistent diagrams to machine learning is related to distances and kernels defined on the space of diagrams, which takes $O(n^2)$ time to compute. Vectrorization schemes such as persistence [images](https://arxiv.org/abs/1507.06217), [landscapes](https://arxiv.org/abs/1501.00179) or [curves](https://arxiv.org/abs/1904.07768) reduce the time to $O(n)$, yet all of this approaches are more or less fixed.

Trainable vectorization allows to learn vector representations of persistence diagrams, optimal w.r.t. the downstream task such as classification or regression. The simplest of such models, [Deep Sets](https://arxiv.org/abs/1703.06114) - $f: (\mathbb{R}^3)^N \rightarrow \mathbb{R}^d$

\begin{equation}
f(\{x_1, \dots, x_N\}) = \rho \left( \sum_{i=1}^N \phi(x_i) \right),
\end{equation}

consists of an encoder $\phi_\theta: \mathbb{R}^3 \rightarrow \mathbb{R}^D$ mapping each diagram point $x_i = (b_i, d_i, h_i)$, with parameters $\theta$ shared between points, a permutation invariant pooling operation $(\cdot): (\mathbb{R}^D)^N \rightarrow \mathbb{R}^D$ to obtain a representation of a diagram at whole (particulary for Deep Sets - sum pooling), and a decoder $\rho: \mathbb{R}^D \rightarrow \mathbb{R}^d$ which further transforms the diagram representation. It was [shown](https://arxiv.org/abs/1904.09378) that certain combinations of encoder/pooling/decoder correspond to the fixed representation schemes of persistence diagrams.

Deep sets encoder vectorizes each single point independently and does not consider the interdependence between points in the diagram. Thus, the self-attention block from the Transformer model which allows to capture those dependencies is a natural plug-in replacement to the encoder $\phi$.

\begin{equation}
\Phi_{ATTN}(\{x_1, \dots, x_N\}) = \left(\frac{(\mathbf{W}_q \mathbf{X})(\mathbf{W}_k \mathbf{X})^T}{\sqrt{D}} \right)\mathbf{W}_v\mathbf{X},
\end{equation}

where $\Phi_{ATTN}: (\mathbb{R}^3)^N \rightarrow (\mathbb{R}^D)^N$.

In [ ]:
def generate_orbit(point_0, r, n=300):
    
    X = np.zeros([n, 2])
    
    xcur, ycur = point_0[0], point_0[1]
    
    for idx in range(n):
        xcur = (xcur + r * ycur * (1. - ycur)) % 1
        ycur = (ycur + r * xcur * (1. - xcur)) % 1
        X[idx, :] = [xcur, ycur]
    
    return X

def generate_orbits(m, rs=[2.5, 3.5, 4.0, 4.1, 4.3], n=300, random_state=None):
    
    # m orbits, each of n points of dimension 2
    orbits = np.zeros((m * len(rs), n, 2))
    
    # for each r
    for j, r in enumerate(rs):

        # initial points
        points_0 = random_state.uniform(size=(m,2))

        for i, point_0 in enumerate(points_0):
            orbits[j*m + i] = generate_orbit(points_0[i], rs[j])
            
    return orbits

In [ ]:
random_state = np.random.RandomState(42)
X_orbit5k = generate_orbits(1000, random_state=random_state)

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(16, 6), dpi=200)
for i in range(2):
    for j in range(5):
        if i==0:
            ax[i,j].set_title("Class {}".format(j+1))
        ax[i,j].scatter(X_orbit5k[j*1000+i,:,0], X_orbit5k[j*1000+i,:,1], s=10)

In [ ]:
X_pc = np.concatenate((X_orbit5k[2000:3000], X_orbit5k[4000:5000]))
y = np.concatenate((np.zeros(1000), np.ones(1000)))

In [ ]:
def conv_pd(diagrams):
    pd = np.zeros((0, 3))

    for k, diagram_k in enumerate(diagrams):
        diagram_k = diagram_k[~np.isinf(diagram_k).any(axis=1)] # filter infs  
        diagram_k = np.concatenate((diagram_k, k * np.ones((diagram_k.shape[0], 1))), axis=1)
        pd = np.concatenate((pd, diagram_k))

    return pd

In [ ]:
X = []
vr = Rips()

for x_pc in tqdm(X_pc):
    diagram = conv_pd(vr.fit_transform(x_pc))
    X.append(diagram)

#### Dataset 

In [ ]:
class Orbit2kDataset(Dataset):
    
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
def collate_fn(data):
    
    tmp_pd, _ = data[0]
    
    n_batch = len(data)
    n_features_pd = tmp_pd.shape[1]
    n_points_pd = max(len(pd) for pd, _ in data)
    inputs_pd = np.zeros((n_batch, n_points_pd, n_features_pd), dtype=float)
    labels = np.zeros(len(data))
    
    for i, (pd, label) in enumerate(data):
        inputs_pd[i][:len(pd)] = pd
        labels[i] = label
    
    return torch.Tensor(inputs_pd), torch.Tensor(labels).long()

#### Model

In [ ]:
class DeepSets(torch.nn.Module):
    def __init__(self, n_in, n_hidden_enc, n_out_enc, n_hidden_dec=16, n_out_dec=2):
        super(DeepSets, self).__init__()
        self.encoder = Encoder(n_in, n_hidden_enc, n_out_enc)
        self.decoder = MLP(n_out_enc, n_hidden_dec, n_out_dec)
        
    def forward(self, X):
        z_enc = self.encoder(X)
        z = self.decoder(z_enc)
        return z
    
class MLP(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super(MLP, self).__init__()
        self.linear1 = Linear(n_in, n_hidden)
        self.linear2 = Linear(n_hidden, n_out)
        
    def forward(self, X):
        X = relu(self.linear1(X))
        X = self.linear2(X)
        return X
    
class Encoder(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super(Encoder, self).__init__()
        self.mlp = MLP(n_in, n_hidden, n_out)
        
    def forward(self, X):
        X = self.mlp(X)
        x = X.mean(dim=1) # aggregation
        return x

#### Train

In [ ]:
%%time
n_repeats = 3
n_epochs = 100
batch_size = 16
lr = 0.001

n_train, n_test = 1600, 400

history = np.zeros((n_repeats, n_epochs, 3))
criterion = CrossEntropyLoss()

dataset = Orbit2kDataset(X, y)

for repeat_idx in range(n_repeats):
    
    # data init
    dataset_train, dataset_test = random_split(dataset, [n_train, n_test])
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    dataloader_test =  DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # model init
    model = DeepSets(n_in=3, n_hidden_enc=16, n_out_enc=8)
    optimizer = Adam(model.parameters(), lr=lr)
    
    print("{:3} {:6} {:6} {:6}".format(repeat_idx, "Loss", "Train", "Test"))
    
    for epoch_idx in range(n_epochs):
        
        # train
        model.train()
        
        loss_epoch = []
        for batch in dataloader_train:
            loss_batch = criterion(model(batch[0]), batch[1])
            loss_batch.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_epoch.append(loss_batch.detach())
        
        loss_epoch_mean = np.array(loss_epoch).mean()
        history[repeat_idx,epoch_idx,0] = loss_epoch_mean
        
        # test
        model.eval()
        
        correct = 0
        for batch in dataloader_train:
            y_hat = model(batch[0]).argmax(dim=1)
            correct += int((y_hat == batch[1]).sum())
        accuracy_train = correct / len(dataloader_train.dataset)
        history[repeat_idx,epoch_idx,1] = accuracy_train

        correct = 0
        for batch in dataloader_test:
            y_hat = model(batch[0]).argmax(dim=1)
            correct += int((y_hat == batch[1]).sum())
        accuracy_test = correct / len(dataloader_test.dataset)
        history[repeat_idx,epoch_idx,2] = accuracy_test
        
        print("{:3} {:.4f} {:.4f} {:.4f}".format(epoch_idx, loss_epoch_mean, accuracy_train, accuracy_test))
    print("\r")

In [ ]:
x = np.arange(n_epochs)
loss_ci1 = history.mean(axis=0)[:,0] - history.std(axis=0)[:,0]
loss_ci2 = history.mean(axis=0)[:,0] + history.std(axis=0)[:,0]
acc_train_ci1 = history.mean(axis=0)[:,1] - history.std(axis=0)[:,1]
acc_train_ci2 = history.mean(axis=0)[:,1] + history.std(axis=0)[:,1]
acc_test_ci1 = history.mean(axis=0)[:,2] - history.std(axis=0)[:,2]
acc_test_ci2 = history.mean(axis=0)[:,2] + history.std(axis=0)[:,2]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 4.5), dpi=200)
fig.suptitle("Persistence diagrams", fontsize=14)
ax[0].set_title("Loss")
ax[1].set_title("Accuracy")
ax[0].set_ylim(0.39, 2.01)
ax[1].set_ylim(0.49, 0.81)
ax[0].plot(history.mean(axis=0)[:,0], c="g")
ax[0].fill_between(x, loss_ci1, loss_ci2, color="g", alpha=0.1)
ax[1].plot(history.mean(axis=0)[:,1], c="r", label="Train")
ax[1].plot(history.mean(axis=0)[:,2], c="b", label="Test")
ax[1].fill_between(x, acc_train_ci1, acc_train_ci2, color="r", alpha=0.1)
ax[1].fill_between(x, acc_test_ci1, acc_test_ci2, color="b", alpha=0.1)
ax[1].legend(loc=4)
plt.show()